In [1]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
SYSTEM_MESSAGE = '''You are building an emotion‑prediction component that, given a character name and a brief situational description, must identify the character’s emotional state and the reason for it.
When given an input of the form:
```
Character:  {character}
Source: {source}
```
your job is to:

1. **Map the Emotion:**  
   - Choose among the 8 primary emotions from Plutchik’s Wheel (joy, trust, fear, surprise, sadness, disgust, anger, anticipation).  
   - For each, assign one of: `"na"` (not applicable), `"low"`, `"medium"`, or `"high"`.

2. **Write a Reason:**  
   - Provide a single sentence explaining why the character feels as you’ve labeled.

## Map the Emotion: Map the target emotion onto the 8 primary emotions from Plutchik’s Wheel
* Joy
* Trust
* Fear
* Surprise
* Sadness
* Disgust
* Anger
* Anticipation

For each emotion, assign one of the following intensities:
* "na" (not applicable)
* "low"
* "medium"
* "high"

Interpret the target emotion (e.g., “authoritative”) in terms of Plutchik’s emotions. For example, you might decide:
* trust: high
* joy: low
* anticipation: medium

## Write a Reason:
Provide a one-sentence rationale ("reason") explaining why the subject (if xReact) or the other person (if oReact) feels the given emotion(s).
ex. “She feels empowered and confident after cutting out social media.”


Return in the following JSON format (no extra keys, no explanation outside the JSON)
{
    "emotion": {
        "joy": "na" | "low" | "medium" | "high",
        "trust": "na" | "low" | "medium" | "high",
        "fear": "na" | "low" | "medium" | "high",
        "surprise": "na" | "low" | "medium" | "high",
        "sadness": "na" | "low" | "medium" | "high",
        "disgust": "na" | "low" | "medium" | "high",
        "anger": "na" | "low" | "medium" | "high",
        "anticipation": "na" | "low" | "medium" | "high"
    },
    "reason": "One sentence explaining why these emotions occur"
}
Only return the JSON'''

USER_TEMPLATE = '''Source: {source}
Character: {character}'''

In [3]:
run_name = "250418-01-qwen2_5-3b-try1"
model_dir = f"weights/{run_name}/best"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", torch_dtype="auto")
model.eval()
model.to("cuda:0")
print("MODEL LOADED")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MODEL LOADED


In [4]:
df = pd.read_csv("data/comet/test_gpt-4.1-nano.csv", sep="\t")
print(df.shape, df.columns)

(1484, 16) Index(['uid', 'original_idx', 'original_src', 'original_relation',
       'original_tgt', 'source', 'character', 'joy', 'trust', 'fear',
       'surprise', 'sadness', 'disgust', 'anger', 'anticipation', 'reason'],
      dtype='object')


# 1. Transformers Generate

In [5]:
row = df.iloc[0]
user_message = USER_TEMPLATE.format(
    source=row['source'],
    character=row['character']
)
assistant_message = json.dumps(
    {
        "emotion": {
            "joy": row['joy'],
            "trust": row['trust'],
            "fear": row['fear'],
            "surprise": row['surprise'],
            "sadness": row['sadness'],
            "disgust": row['disgust'],
            "anger": row['anger'],
            "anticipation": row['anticipation']
        },
        "reason": row['reason']
    }
)

messages = [
    {"role": "system", "content": SYSTEM_MESSAGE},
    {"role": "user", "content": user_message},
    {"role": "assistant", "content": assistant_message}
]

In [6]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Directly use generate() and tokenizer.decode() to get the output.
# Use `max_new_tokens` to control the maximum output length.
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)

In [7]:
print(tokenizer.decode(
    generated_ids[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
))

system
You are building an emotion‑prediction component that, given a character name and a brief situational description, must identify the character’s emotional state and the reason for it.
When given an input of the form:
```
Character:  {character}
Source: {source}
```
your job is to:

1. **Map the Emotion:**  
   - Choose among the 8 primary emotions from Plutchik’s Wheel (joy, trust, fear, surprise, sadness, disgust, anger, anticipation).  
   - For each, assign one of: `"na"` (not applicable), `"low"`, `"medium"`, or `"high"`.

2. **Write a Reason:**  
   - Provide a single sentence explaining why the character feels as you’ve labeled.

## Map the Emotion: Map the target emotion onto the 8 primary emotions from Plutchik’s Wheel
* Joy
* Trust
* Fear
* Surprise
* Sadness
* Disgust
* Anger
* Anticipation

For each emotion, assign one of the following intensities:
* "na" (not applicable)
* "low"
* "medium"
* "high"

Interpret the target emotion (e.g., “authoritative”) in terms of Plu

# 2. Outlines Generate

In [8]:
import outlines
from outlines import models, generate

from pydantic import BaseModel
from typing import Optional, Dict, Any, List
from enum import Enum

In [9]:
class RelationshipStatus(str, Enum):
    na = "na"
    low = "low"
    medium = "medium"
    high = "high"
    
class EmotionLabel(BaseModel):
    joy: RelationshipStatus
    trust: RelationshipStatus
    fear: RelationshipStatus
    surprise: RelationshipStatus
    sadness: RelationshipStatus
    disgust: RelationshipStatus
    anger: RelationshipStatus
    anticipation: RelationshipStatus
    
    # class Config:
    #     extra = Extra.forbid
    #     use_enum_values = True
        
class EntryResult(BaseModel):
    emotion: EmotionLabel
    reason: str

In [11]:
outlines_model = models.Transformers(model, tokenizer,)

In [12]:
generator = outlines.generate.json(outlines_model, EntryResult)

In [13]:
prediction = generator(text)

print(repr(prediction))

EntryResult(emotion=EmotionLabel(joy=<RelationshipStatus.medium: 'medium'>, trust=<RelationshipStatus.high: 'high'>, fear=<RelationshipStatus.na: 'na'>, surprise=<RelationshipStatus.na: 'na'>, sadness=<RelationshipStatus.na: 'na'>, disgust=<RelationshipStatus.na: 'na'>, anger=<RelationshipStatus.na: 'na'>, anticipation=<RelationshipStatus.low: 'low'>), reason='Maria feels happy and satisfied because her friend showed affection, strengthening their bond.')
